In [1]:
import sys
import ee 
%run Sentinel2b.ipynb

from authentication import authenticate
authenticate()

modulename = 'ee'
if modulename not in sys.modules:
    # import GEE and Authenticate, token or log in will be asked from web browser
    import ee
    # TO DO : UNCOMMENT THIS IF IT IS THE FIRST TIME AUTHINTICATE GEE ON THIS MACHINE
    # ee.Authenticate()
    ee.Initialize()
# else:
    # google earth engine already imported and authenticated


modulename = 'ipynb_MapVisualisation'
if modulename not in sys.modules:
    %run MapVisualisation.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_MapVisualisation'] = None
# else
    # Utils modules has already been loaded somewhere else


modulename = 'ipynb_Sentinel2b'
if modulename not in sys.modules:
    %run Sentinel2b.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_Sentinel2b'] = None
# else
    # Utils modules has already been loaded somewhere else


Utils imported
Masks class imported
Sentinel2b class imported
MapVisualisation class imported - uses folium library
Sentinel2b class imported


In [2]:
from indices import INDEX_NAMES

In [3]:
# Area of Interest
AOI = ee.Geometry.Polygon(
        [[[-2.67404756802236, 41.99472813320794],
          [-2.67404756802236, 41.8107480842418],
          [-2.36643038052236, 41.8107480842418],
          [-2.36643038052236, 41.99472813320794]]])

# cloudmask01 = getCloudMask(median)

START_DATE = '2019-08-15'
END_DATE = '2019-12-31'



masks = {}#,'forestMask': {'buffer':10, 'startDate':'2000-01-01', 'endDate':'2022-12-31',}} # # , 'lmask': 30,
# index_names = INDEX_NAMES[0:2]
index_names = INDEX_NAMES[:]

#printAvailableIndicesS2()

sentinel2Info = {
    "identifier": 'sentinel-2',                                              
    "collection": "COPERNICUS/S2_SR_HARMONIZED",
    #"launchDate" : "2015-05-23",
    #"retirmentDate":todayDate, #not retired yet
    "startDate"   : '2019-01-01',
    "endDate"     : '2019-12-31',
    #"added"     : False,
    "clouds"    : 50,
    "selectedBand" : ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'],
    "selectedIndices": index_names #INDEX_NAMES
} 


s2 = Sentinel2(AOI, sentinel2Info, masks)


# s2.byMonth(2020)

#median = s2.getMedian()
#bandNames = median.bandNames()
#print('Band names medianS2: ', bandNames.getInfo())

# Code used for testing cloud masks - Cloud masking used by default so no need to worry about this code
#med = s2.getCollection().first()
#clouds = med.select('clouds').selfMask()
#shadows = med.select('shadows').selfMask()
#dark_pixels = med.select('dark_pixels')  # .selfMask()
#probability = med.select('probability')
#cloudmask = med.select('cloudmask')
#cloudmaskMasked = cloudmask  # .selfMask()
#cloud_transform = med.select('cloud_transform')

# Selecting RGB bands for visualisation purposes
#medianS2 = median.select(['B4', 'B3', 'B2'])


# Calculate NDVI on the median image
img = s2.getFirst()
imgBands = img.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])
# TO DO: Select one at a time due to type inconsistency
# imgindices = img.select(["ndvi"])
imgindices = img.select(index_names)

# export image  to GEE - adding maxPixels is a bad habit, used carefully
task = ee.batch.Export.image.toDrive(**{
    'image': imgindices,
    'description': 'imgindices',
    'folder': 'earth_engine_demos',
    'scale': 10,
    'region': AOI.getInfo()['coordinates'],
    'maxPixels': 1549491660
})
task.start()

# # export image  to GEE - adding maxPixels is a bad habit, used carefully
# task = ee.batch.Export.image.toDrive(**{
#     'image': imgBands,
#     'description': 'imgbands',
#     'folder': 'earth_engine_demos',
#     'scale': 10,
#     'region': AOI.getInfo()['coordinates'],
#     'maxPixels': 1549491660
# })
# task.start()



selectedBandsNIndices: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'arvi', 'bi', 'bi2', 'ci', 'dvi', 'gemi', 'gndvi', 'ipvi', 'ireci', 'mcari', 'mndwi', 'msavi2', 'mtci', 'dni45', 'ndpi', 'ndti', 'ndwi', 'ndwi2', 'pssra', 'reip', 'ri', 'ndvi', 'savi', 'tndvi', 'vi']


/home/finley/Work/RDS/projects/greensight/.venv/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for UMD/hansen/global_forest_change_2022_v1_10! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2022_v1_10

  warnings.warn(warning, category=DeprecationWarning)


In [6]:
%run MapVisualisation.ipynb
# These are the coordinates of Spain
my_map = folium.Map(location=[42, -3], zoom_start=11)
# Add custom basemaps
# basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)


def update_cld_shdw_mask(img):
    print('test')


# ranges = {
#     "arvi": (0, 1),
#     "bi": (0, 2500),
#     "bi2": (0, 2500),
#     "ci": (0, 1),
#     "dvi": (0, 2500),
# }
ranges = {
    "arvi": (-1, 1),  # Atmospherically Resistant Vegetation Index
    "bi": (0, 2500),  # Brightness Index (scaled reflectance)
    "bi2": (0, 2500),  # Brightness Index 2 (alternative brightness measure)
    "ci": (0, 1),  # Chlorophyll Index (varies depending on formulation)
    "dvi": (-1000, 5000),  # Difference Vegetation Index (NIR - Red)
    "gemi": (-2500000, 2500000),  # Global Environment Monitoring Index
    "gndvi": (-1, 1),  # Green Normalized Difference Vegetation Index
    "ipvi": (0, 1),  # Infrared Percentage Vegetation Index
    "ireci": (-2500, 2500),  # Infrared Ratio Vegetation Index
    "mcari": (0, 10),  # Modified Chlorophyll Absorption Ratio Index
    "mndwi": (-1, 1),  # Modified Normalized Difference Water Index
    "msavi2": (0, 1),  # Modified Soil-Adjusted Vegetation Index 2
    "mtci": (0, 10),  # Meris Terrestrial Chlorophyll Index
    "dni45": (-1, 1),  # Drought Normalized Index 45
    "ndpi": (-1, 1),  # Normalized Difference Plant Index
    "ndti": (-1, 1),  # Normalized Difference Tillage Index
    "ndwi": (-1, 1),  # Normalized Difference Water Index
    "ndwi2": (-1, 1),  # Normalized Difference Water Index 2
    "pssra": (0, 10),  # Plant Stress Severity Ratio
    "reip": (680, 750),  # Red Edge Inflection Point (wavelength in nm)
    "ri": (0, 1),  # Ratio Index
    "ndvi": (-1, 1),  # Normalized Difference Vegetation Index
    "savi": (0, 10),  # Soil-Adjusted Vegetation Index
    "tndvi": (0, 1),  # Transformed Normalized Difference Vegetation Index
    "vi": (0, 10),  # General Vegetation Index
}
# Add Land Mask to the map
# my_map.add_ee_layer(imgBands, {
                    # 'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'median')
for name in index_names:
# for name in ["vi", "tndvi"]:
    ind = img.select(name)
    my_map.add_ee_layer(ind, {"min": ranges[name][0], "max": ranges[name][1]}, name)
my_map.add_ee_layer(AOI, {}, 'AOI')



# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
# plugins.Fullscreen().add_to(my_map):w


# Display the map.
display(my_map)


MapVisualisation class imported - uses folium library


In [ ]:
ind.get("ndvi")

In [41]:
mean_band_value = ind.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=AOI,
    scale=30
)

In [47]:
mean_band_value.getInfo()

{'ndvi': 4.399667137148222}

In [31]:
print(out.get("B4"))

ee.ComputedObject({
  "functionInvocationValue": {
    "functionName": "Element.get",
    "arguments": {
      "object": {
        "functionInvocationValue": {
          "functionName": "Image.sampleRectangle",
          "arguments": {
            "image": {
              "functionInvocationValue": {
                "functionName": "Collection.first",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.map",
                      "arguments": {
                        "baseAlgorithm": {
                          "functionDefinitionValue": {
                            "argumentNames": [
                              "_MAPPING_VAR_0_0"
                            ],
                            "body": {
                              "functionInvocationValue": {
                                "functionName": "Image.updateMask",
                                "arguments": {
  